### Load document into index

In [ ]:
import sys
import os
from dotenv import load_dotenv

load_dotenv("../src/utils/.env")
sys.path.append(os.path.abspath("../src"))

from pipeline.loader import load_pdf_chunks
from pipeline.indexer import index_documents

pdf_path = "../sample_data/Certificado_Senen Fernandez_UPM.pdf"

chunks = load_pdf_chunks(pdf_path)
index_documents(chunks, persist_directory="../src/db", collection_name="knowledge_base" )

### Load image Document into Index

In [ ]:
# 📦 Install required packages (only run once)
!pip install openai pillow python-dotenv

# 🧠 Import libraries
import os
import sys
import re
import base64
import openai
from PIL import Image
from typing import List
from dotenv import load_dotenv
from langchain.schema import Document

# 🌍 Load environment variables (expects OPENAI_API_KEY in .env)
load_dotenv("../src/utils/.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

# 📂 Load helper functions from your pipeline
sys.path.append(os.path.abspath("../src"))
from pipeline.indexer import index_documents

# 🧠 Generate caption using GPT-4V
def caption_image_with_gpt4v(image_path: str) -> str:
    with open(image_path, "rb") as img_file:
        img_base64 = base64.b64encode(img_file.read()).decode("utf-8")

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that describes images accurately."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Describe this image in one or two sentences."},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
                ]
            }
        ],
        max_tokens=100
    )
    return response.choices[0].message.content.strip()

# 🧾 Extract page number from filename
def extract_page_number(filename: str) -> int:
    match = re.search(r'page(\d+)', filename)
    return int(match.group(1)) if match else -1

# 🗂️ Process images in a folder (or single image)
def process_images(folder_path: str, specific_image: str = None) -> List[Document]:
    if specific_image:
        image_files = [os.path.join(folder_path, specific_image)]
    else:
        image_files = [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]

    documents = []
    for image_path in image_files:
        filename = os.path.basename(image_path)
        page = extract_page_number(filename)
        print(f"📄 Processing: {filename} (page {page})")

        try:
            caption = caption_image_with_gpt4v(image_path)
        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")
            continue

        if not caption.strip():
            print(f"⚠️ Empty caption for {filename}. Skipping.")
            continue
        print(caption)
        doc = Document(
            page_content=caption,
            metadata={
                "page_number": page,
                "image_file": filename,
                "source": os.path.splitext(filename)[0].split("_extracted_images")[0] + ".pdf",  # match PDF source
                "type": "image",
                "path": image_path  # helpful if showing image
            }
        )
        documents.append(doc)

    return documents

# ✅ Index image documents into ChromaDB
def index_image_documents(folder_path: str, specific_image: str = None, collection_name: str = "knowledge_base"):
    image_docs = process_images(folder_path, specific_image)

    valid_docs = [doc for doc in image_docs if doc.page_content and doc.page_content.strip()]
    if not valid_docs:
        print("⚠️ No valid documents to index.")
        return

    index_documents(valid_docs, persist_directory="../src/db", collection_name=collection_name)
    print(f"✅ Indexed {len(valid_docs)} image documents into '{collection_name}' collection.")


In [ ]:

# 🔍 Example usage:
# To ingest all images in the folder
# index_image_documents("../sample_data/pdf_with_images_extracted_images")

# To ingest a single image
index_image_documents("../sample_data/pdf_with_images_extracted_images", specific_image="image_page1_1.jpeg")


### Index Form (Formulario de TIE)

In [ ]:
import os
import json
import openai
import fitz  # pymupdf
from dotenv import load_dotenv
from typing import Dict

# 🌱 Cargar variables de entorno
load_dotenv("../src/utils/.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

# 📄 Leer texto del PDF usando PyMuPDF
def extract_text_from_pdf(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text() + "\n"
    doc.close()
    return full_text.strip()

# 🤖 Llamar a GPT para extraer campos del formulario
def extract_structured_info(text: str, pdf_filename: str) -> Dict:
    prompt = f"""
Eres un sistema de extracción de información para formularios oficiales.

Extrae los siguientes campos del texto proporcionado. Si no puedes encontrar alguno, escribe "N/A".

Campos:
- numero_pasaporte
- nie
- apellido_1
- apellido_2
- nombre
- sexo
- fecha_nacimiento
- lugar_nacimiento
- pais_nacimiento
- nacionalidad
- estado_civil
- nombre_padre
- nombre_madre
- domicilio_espana
- localidad
- codigo_postal
- provincia
- telefono_movil
- email

Texto del documento:
{text}

Devuelve un objeto de JSON con los campos solicitados
    """
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Eres un extractor de información estructurada de documentos."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=1000
    )

    # Extrae solo el contenido entre las primeras llaves {}
    
    content = response.choices[0].message.content
    start = content.find("{")
    end = content.rfind("}")
    if start != -1 and end != -1 and end > start:
        content = content[start:end+1]
    else:
        print("⚠️ No se encontraron llaves en la respuesta. Usando contenido completo.")
    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        print("⚠️ Error al parsear JSON. Respuesta del modelo:")
        print(content)
        data = {"error": "No se pudo extraer información"}

    # Añadir metadatos del documento
    data["source_pdf"] = pdf_filename
    return data

# 🧾 Función principal para procesar un formulario PDF
def process_form_pdf(pdf_path: str) -> Dict:
    pdf_text = extract_text_from_pdf(pdf_path)
    filename = os.path.basename(pdf_path)
    structured_data = extract_structured_info(pdf_text, filename)
    return structured_data

# 💾 Guardar resultados en JSON
def save_json(data: Dict, output_path: str):
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"✅ Datos guardados en {output_path}")


In [30]:
# Ruta al formulario TIE escaneado en PDF
pdf_path = "../raw_data/form.pdf"
output_json_path = "../raw_data/form.json"

# Procesar el PDF y extraer información estructurada
datos_tie = process_form_pdf(pdf_path)

# Guardar en JSON
save_json(datos_tie, output_json_path)

# Mostrar resultados
from pprint import pprint
pprint(datos_tie)


✅ Datos guardados en ../raw_data/form.json
{'apellido_1': 'FERNANDEZ',
 'apellido_2': 'GARCIA',
 'codigo_postal': '12345',
 'domicilio_espana': 'Alcazar de Sevilla, 1, 3',
 'email': 'eugenio@gmail.com',
 'estado_civil': 'Soltero',
 'fecha_nacimiento': '01/01/2000',
 'localidad': 'Sevilla',
 'lugar_nacimiento': 'Madrid',
 'nacionalidad': 'Española',
 'nie': 'X12345678A',
 'nombre': 'EUGENIO',
 'nombre_madre': 'Rigoberta',
 'nombre_padre': 'Eustaquio',
 'numero_pasaporte': '111222333X',
 'pais_nacimiento': 'España',
 'provincia': 'Andalucía',
 'sexo': 'H',
 'source_pdf': 'form.pdf',
 'telefono_movil': '666111111'}
